In [1]:
import pandas as pd
import numpy as np
import tweepy
import nltk
import re
from time import sleep
import io
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import itertools
from sentence_transformers import SentenceTransformer, util



In [2]:
min_tweet_length = 10


topics_en = [
    "We should stop subsidizing meat",
    "We should subsidize green nutrition",
    "We have to increase production of meat",
    "We should exempt meat production from carbon taxes",
    "We should add a carbon tax to food production",
    "Meat alternatives should be encouraged",
    "Plant based should be encouraged",
    "Meat alternatives should be invested in",
    "Plant based should be invested in",
    "Meat alternatives should be subsidized",
    "Plant based should be subsidized",
    "We should introduce meatless mondays",
    "Vegetarian and vegan diets should be encouraged",
    "Vegetarian and vegan diets should be discouraged",
    "We should subsidize fruits and vegetables",
    "We should encourage more fruits and vegetable consumption",
    "We should discourage fruits and vegetable consumption",
] 

topics_da = [
    "Vi bør stoppe med at subsidiere kød",
    "Vi bør subsidiere grøn ernæring",
    "Vi skal øge produktionen af kød",
    "Vi bør fritage kød produktion fra co2 afgifter",
    "Vi bør tilføje en co2 afgift til fødevare produktionen",
    "Alternativer til kød bør fremmes og støttes",
    "Plantebaseret bør fremmes og støttes",
    "Der bør investeres i kødalternativer",
    "Der bør investeres i plantebaseret",
    "Alternativer til kød bør subsidieres",
    "Plantebaseret bør subsidieres",
    "Der bør opfordres til vegetarisk og vegansk kost",
    "Vegetarisk og vegansk kost bør frarådes",
    "Vi bør subsidiere frugt og grøntsager",
    "Vi bør fremme forbruget af frugt og grøntsager",
    "Vi bør fraråde forbrug af frugt og grøntsager"
]

tweet_df = pd.read_csv('scraped_tweets_new.csv')

In [4]:
def pre(string, lang):
    stemmer = SnowballStemmer('english' if lang == 'en' else 'danish')
    stops = set(stopwords.words('english' if lang == 'en' else 'danish'))
    
    words = word_tokenize(re.sub('[^a-zA-Z]', ' ', string.lower().strip()))                        
    meaningful_words = [w for w in words if not w in stops]   
    return ' '.join([stemmer.stem(w) for w in meaningful_words])

def filter_tweets(df):
    return df[df.tweet.str.split().str.len() > min_tweet_length]

def score_tfidf(df, topic, lang):
    df = filter_tweets(df).copy()
    df['score'] = [0]*len(df) 

    corpus = list(map(lambda x: pre(x, lang), df[df.lang == lang].tweet.values))

    vectorizer = TfidfVectorizer()
    tfidf_mtx = vectorizer.fit_transform(corpus)
    topic_mtx = vectorizer.transform([pre(topic, lang)])

    cos_score = cosine_similarity(tfidf_mtx, topic_mtx)

    df.loc[df.lang == lang, ['score']] = cos_score
    return df

scored = score_tfidf(tweet_df, topics_en[9], 'en')

top = scored.sort_values(by=['score'], ascending=False)

In [20]:
def print_df_scores(df):
    top = df.sort_values(by=['score'], ascending=False)
    i = 0
    for r in top.head(i+10).iloc[i:].iloc:
        print('Topic:', r.topic, 'Tweet:', r.tweet, 'Score:', r.score)
        print('='*120)

In [22]:

def qa_model_score(df, topics, dot_score = True):    
    df = df.copy()
    df['score'] = [0]*len(df) 
    
    #Load the model
    model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

    docs = df.tweet.values
    doc_emb = model.encode(docs)

    
    score_dfs = [] 
    for topic in topics:
    
        df['score'] = [0]*len(df) 
        df['topic'] = [topic]*len(df)
    
        #Encode query and documents
        query_emb = model.encode(topic)
        
        if dot_score:
            #Compute dot score between query and all document embeddings
            scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
        else:
            scores = cosine_similarity(query_emb, doc_emb)[0].tolist()
            
        #Combine docs & scores
        #doc_score_pairs = list(zip(docs, scores))
        df['score'] = scores
        score_dfs.append(df.copy())

        #Sort by decreasing score
        #doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    return score_dfs
        


In [23]:
scores = qa_model_score(tweet_df, topics_en)

In [30]:
print_df_scores(scores[13])

Topic: Vegetarian and vegan diets should be discouraged Tweet: <MENTION> not vegan VEGETARIAN Score: 0.8436784744262695
Topic: Vegetarian and vegan diets should be discouraged Tweet: all of your dietary arguments against veganism are utterly pathetic. Score: 0.8180283904075623
Topic: Vegetarian and vegan diets should be discouraged Tweet: <MENTION> No vegetarian or vegan option? Score: 0.814056396484375
Topic: Vegetarian and vegan diets should be discouraged Tweet: <MENTION> <MENTION> Vegetarian is enough vegan is extreme Score: 0.8133179545402527
Topic: Vegetarian and vegan diets should be discouraged Tweet: <MENTION> Cos your vegan? Vegetarian? Score: 0.8091829419136047
Topic: Vegetarian and vegan diets should be discouraged Tweet: I might be able to do a vegetarian diet but vegan?! No Score: 0.8066393733024597
Topic: Vegetarian and vegan diets should be discouraged Tweet: <MENTION> Vegetarian with vegan options. Score: 0.8025890588760376
Topic: Vegetarian and vegan diets should be d

In [51]:
sample = tweet_df[tweet_df.].sample(25)

In [52]:
sample['topic'] = np.random.choice(topics_en, len(sample))
sample['score'] = [0]*len(sample)

In [53]:
def random_top_samples(dfs, samples, top = 100):
    top_dfs = []
    for df in dfs:
        top_dfs.append(df.sort_values(by=['score'], ascending=False)[:top])
    return pd.concat(top_dfs).sample(samples)

In [54]:
sample = pd.concat([sample, random_top_samples(scores, 25)])

In [55]:
sample

,tweet,id,lang,topic,score
7242,<MENTION> <MENTION> <MENTION> What? You get th...,1.510900e+18,en,We should discourage fruits and vegetable cons...,0.000000
10848,"<MENTION> Nope, they haven’t for anyone. All t...",1.511696e+18,en,We should introduce meatless mondays,0.000000
4806,<MENTION> Two of these “dumb” policies enacted...,1.510039e+18,en,We should stop subsidizing meat,0.000000
28324,Just played: SCAD Atlanta Radio - Jelly Beans,1.511980e+18,en,We should introduce meatless mondays,0.000000
16379,<MENTION> The only things I've found that work...,1.511841e+18,en,We have to increase production of meat,0.000000
4345,"A change toward avoiding buying ( industrial, ...",1.510557e+18,en,We should encourage more fruits and vegetable ...,0.000000
26997,1/2lb turkey burger on a ciabatta bun with her...,1.512010e+18,en,Plant based should be invested in,0.000000
3156,Even though it's bad for the climate the US ne...,1.511120e+18,en,Vegetarian and vegan diets should be discouraged,0.000000
24315,<MENTION> <MENTION> Put it inside a fruit chew...,1.512013e+18,en,We should subsidize fruits and vegetables,0.000000
12052,<MENTION> Great source of government revenues ...,1.509176e+18,en,Plant based should be subsidized,0.000000
